In [9]:
import pandas as pd

def parse_measurement_block(lines, start_idx):
    """
    Parse one measurement 'block' that starts with something like:
      Measurement,Units,1,2,3,4,5,...
    and is followed by one or more lines like:
      Rotor Position theta_r,[ degrees ],0,1,2,3,...
      Torque T_e,[ mN*m ],-49.3,-48.7,-60.2,...
    until the next 'Measurement,Units' line or a blank line.

    Returns:
      (lines_consumed, block_data_dict)
        where block_data_dict = { "MeasurementName (unit)": [...list of floats...] }
    """

    header_line = lines[start_idx].strip()
    header_parts = [p.strip() for p in header_line.split(',')]
    # header_parts[0] = "Measurement", header_parts[1] = "Units", rest are numeric column headers (e.g. 1,2,3..)

    block_data = {}  # measurement_name -> list of float values
    line_idx = start_idx + 1

    while line_idx < len(lines):
        row = lines[line_idx].strip()
        # Stop if blank or we've hit next "Measurement,Units" block
        if (not row) or row.startswith("Measurement,Units"):
            break

        parts = [p.strip() for p in row.split(',')]
        if len(parts) < 2:
            break  # skip weird/short lines

        meas_name_raw = parts[0].strip('"')
        meas_unit_raw = parts[1].strip('"[] ')

        # Build a final column name, e.g. "Rotor Position theta_r (degrees)"
        col_name = meas_name_raw
        if meas_unit_raw:
            col_name += f" ({meas_unit_raw})"

        # Convert all subsequent items to float where possible
        numeric_vals = []
        for val in parts[2:]:
            val = val.strip()
            if not val:
                numeric_vals.append(None)
            else:
                try:
                    numeric_vals.append(float(val))
                except ValueError:
                    # If it can’t parse as float, just store raw text
                    numeric_vals.append(val)

        # Append to block_data
        if col_name not in block_data:
            block_data[col_name] = []
        block_data[col_name].extend(numeric_vals)

        line_idx += 1

    lines_consumed = line_idx - start_idx
    return lines_consumed, block_data


def parse_one_table(table_lines):
    """
    Given a list of lines for a single table, parse all measurement blocks
    and return a Pandas DataFrame with columns for each measurement.
    """
    idx = 0
    final_data = {}  # measurement -> list of values

    while idx < len(table_lines):
        line = table_lines[idx].strip()
        # Skip empty lines
        if not line:
            idx += 1
            continue

        # If this line starts a measurement block
        if line.startswith("Measurement,Units"):
            consumed, block_data = parse_measurement_block(table_lines, idx)
            # Merge block_data into final_data
            for k, arr in block_data.items():
                if k not in final_data:
                    final_data[k] = arr
                else:
                    final_data[k].extend(arr)
            idx += consumed
        else:
            idx += 1

    # Build final DataFrame
    df = pd.DataFrame(final_data)
    return df


def parse_csv_tables(file_path):
    """
    Reads the entire CSV (with possibly multiple "Task X" tables) via pandas.read_csv,
    splits them into table-specific chunks, and returns {table_name: DataFrame}.
    """
    # 1) Read raw CSV into a DataFrame with no header, preserving text in each cell.
    #    Use engine='python' so we can handle unusual CSV structures & blank lines.
    df_raw = pd.read_csv(file_path, header=None, keep_default_na=False, engine='python')

    # 2) Convert each row to a single "comma-joined" line of text,
    #    ignoring trailing empty cells.
    def row_to_csv_line(row):
        # e.g. row might be [ 'Measurement', 'Units', '1', '2', '', '', ... ]
        # We'll keep only non-empty strings up to the last non-empty cell,
        # then join with commas so we can detect line.startswith("Task...")
        # or line.startswith("Measurement,Units").
        row_strs = [str(x) for x in row if str(x).strip() != '']
        return ",".join(row_strs)

    lines = [row_to_csv_line(r) for r in df_raw.values.tolist()]

    tables = {}
    current_table_name = None
    current_table_lines = []

    def flush_current_table():
        if current_table_name and current_table_lines:
            df_table = parse_one_table(current_table_lines)
            tables[current_table_name] = df_table

    for line in lines:
        strip_line = line.strip()
        # Detect "start of a new table" by lines that begin with "Task" or "Task 1-A," etc.
        if strip_line.startswith('"Task') or strip_line.startswith('Task'):
            # flush the previous table
            flush_current_table()
            current_table_name = strip_line.strip('"')
            current_table_lines = []
        else:
            # Accumulate lines for current table
            if current_table_name is not None:
                current_table_lines.append(line)

    # flush the last table
    flush_current_table()

    return tables



file_path = "task_1.csv"  # point this to your actual CSV file
parsed_tables = parse_csv_tables(file_path)

for name, df in parsed_tables.items():
    print(f"===== TABLE: {name} =====")
    display(df)
#
# That’s it! Each table is now in a separate DataFrame in `parsed_tables`.

ValueError: All arrays must be of the same length

In [3]:
table1 = tables["Table_1"]
table1

1,Measurement,Units,1.0,2.0,3.0,4.0,5,6.0,7.0,8,9,10.0,NaN
0,Rotor Position theta_r,degrees,1.0,2.0,3.0,4.0,5,6.0,7.0,8,9,10.0,NaN
1,Torque T_e,mN*m,0.2,13.6,27.0,28.6,23,7.0,-6.0,-16.2,-34,-49.0,NaN
2,Measurement,Units,11.0,12.0,13.0,14.0,15,16.0,17.0,18,19,20.0,NaN
3,Rotor Position theta_r,degrees,11.0,12.0,13.0,14.0,15,16.0,17.0,18,19,20.0,21
4,Torque T_e,mN*m,-52.0,-48.0,-37.8,-19.5,-7.5,13.6,26.2,29.6,26.8,12.0,-4.4
